In [ ]:
## References:
# https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes

In [1]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Mounted at /content/MyDrive/


In [2]:
%cd /content/MyDrive/MyDrive/DL_Project

/content/MyDrive/.shortcut-targets-by-id/1mJgNU1aJtlW2TXPFO2e4TWL1mJ2AVebd/DL_Project


In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
from tqdm import tqdm 
import torchvision.transforms as transforms

In [4]:
BASE_DIR = './NewDataset/'

In [5]:
# curating annotations into category wise files
data = []
for i in tqdm(range(1,6)):
  BASE_DIR = f"./newestDataset/NewDataset/Len{i}Char/data/"
  for file in tqdm(os.listdir(BASE_DIR)):
    data.append(
        [file, str(file.split('_')[0]), i, 'Num']
    )


100%|██████████| 5/5 [02:20<00:00, 28.19s/it]


In [6]:
df = pd.DataFrame(data, columns = ['file', 'text', 'length', 'category'], dtype=str)
df.head()
df.to_csv('./newestDataset/Num.csv', index=False)

In [9]:
# df[df['text'].str.len()==0]

,file,text,length,category


In [5]:
# # curating annotations into category wise files
# data = []
# # for i in tqdm(range(1,6)):
# #   BASE_DIR = f"./Dataset/Num/Len{i}Char/"
# for file in tqdm(os.listdir(BASE_DIR)):
#     if 'Num' in file:
#       continue
#     data.append(
#         [file, str(file[:5]), 5, 'Num']
#     )
# df = pd.DataFrame(data, columns = ['file', 'text', 'length', 'category'], dtype=str)
# df.head()


100%|██████████| 3002/3002 [00:00<00:00, 910631.42it/s]


,file,text,length,category
0,86072_62632c84-f943-4a94-8119-5ba47095dc97.png,86072,5,Num
1,74358_e66955b8-cbf5-4177-a87a-2d5ab749ce49.png,74358,5,Num
2,81709_59258333-1619-4bff-80ea-7e2f98e1c52b.png,81709,5,Num
3,70362_24f6e1f6-7b7d-43d8-8c15-0fce57871787.png,70362,5,Num
4,79346_d17155b3-558f-4aaf-ab64-2be7438b113e.png,79346,5,Num


In [ ]:
df.shape

(3002, 4)

In [6]:
df.to_csv('./Num.csv', index=False)

In [ ]:
df = pd.read_csv(f'{BASE_DIR}Num2.csv') 
df.head()

,file,text,length,category
0,81496_e6921aa4-14c8-4c35-a63d-df37c7b59c92.png,81496,5,Num
1,86072_62632c84-f943-4a94-8119-5ba47095dc97.png,86072,5,Num
2,74358_e66955b8-cbf5-4177-a87a-2d5ab749ce49.png,74358,5,Num
3,81709_59258333-1619-4bff-80ea-7e2f98e1c52b.png,81709,5,Num
4,70362_24f6e1f6-7b7d-43d8-8c15-0fce57871787.png,70362,5,Num


In [ ]:
class CaptchaDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(f"{root_dir}{annotation_file}")
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index]
        img = Image.open(
            # os.path.join( self.root_dir, img_id['category'], f"Len{img_id['length']}Char", img_id['file'])
            os.path.join( self.root_dir, img_id['file'])
          ).convert("L")
        y_label = (img_id['length'], img_id['text'])

        if self.transform is not None:
            img = self.transform(img)
  
        return (img, y_label)

In [ ]:
transform = transforms.Compose(
        [
            # transforms.Resize((356, 356)),
            # transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5)),
        ]
    )

In [ ]:
BATCH = 16

In [ ]:
dataset = CaptchaDataset(BASE_DIR, "Num.csv", transform = transform)
train_set, validation_set = torch.utils.data.random_split(dataset, [2500,500])

train_loader = DataLoader(dataset = train_set, shuffle = True, batch_size = BATCH)
validation_loader = DataLoader(dataset = validation_set, shuffle = True, batch_size = BATCH)

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(type(labels[1]))

<class 'torch.Tensor'>
torch.Size([16, 1, 50, 135])
<class 'torch.Tensor'>
